# Module 02: Doping and the PN Junction

**Electricity & Semiconductors Course**

---

In Module 01 we learned that silicon is a semiconductor — its conductivity sits between conductors and insulators. Now we explore *why* silicon is so useful: we can precisely control its conductivity by adding tiny amounts of impurities. This process is called **doping**, and it is the foundation of every semiconductor device.

By the end of this notebook you will understand:
- How the silicon crystal lattice is structured
- What N-type and P-type doping do at the atomic level
- What happens when P-type meets N-type silicon
- How forward and reverse bias control current through a PN junction
- The exponential IV relationship that governs diode behavior

## Concept — The Silicon Crystal Lattice

Silicon (Si) has **4 valence electrons**. In a pure silicon crystal, every atom forms **covalent bonds** with 4 neighboring silicon atoms, sharing one electron with each neighbor. This creates a stable, repeating **diamond cubic** structure.

Key facts:
- Each bond consists of **2 shared electrons**
- All valence electrons are locked in bonds — very few are free to move
- At absolute zero, pure silicon is a perfect insulator
- At room temperature, thermal energy frees some electrons, giving silicon its modest conductivity

This is **intrinsic silicon** — no impurities, roughly equal numbers of free electrons and holes.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

def draw_lattice(ax, title, highlight_pos=None, highlight_type=None, extra_carrier=None):
    """Draw a simplified 2D silicon crystal lattice."""
    ax.set_xlim(-0.5, 4.5)
    ax.set_ylim(-0.5, 4.5)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.axis('off')
    
    # Draw atoms on a grid
    for i in range(5):
        for j in range(5):
            if highlight_pos and (i, j) == highlight_pos:
                color = '#e74c3c' if highlight_type == 'B' else '#2ecc71'
                label = highlight_type
            else:
                color = '#3498db'
                label = 'Si'
            circle = plt.Circle((i, j), 0.22, color=color, ec='black', lw=1.5, zorder=5)
            ax.add_patch(circle)
            ax.text(i, j, label, ha='center', va='center', fontsize=8,
                    fontweight='bold', color='white', zorder=6)
    
    # Draw bonds
    for i in range(5):
        for j in range(5):
            if i < 4:
                ax.plot([i + 0.22, i + 0.78], [j, j], 'k-', lw=1.5, zorder=1)
                ax.plot(i + 0.4, j + 0.05, 'ko', markersize=3, zorder=2)
                ax.plot(i + 0.6, j - 0.05, 'ko', markersize=3, zorder=2)
            if j < 4:
                ax.plot([i, i], [j + 0.22, j + 0.78], 'k-', lw=1.5, zorder=1)
                ax.plot(i + 0.05, j + 0.4, 'ko', markersize=3, zorder=2)
                ax.plot(i - 0.05, j + 0.6, 'ko', markersize=3, zorder=2)
    
    if extra_carrier:
        cx, cy, carrier_label, carrier_color = extra_carrier
        ax.annotate(carrier_label, xy=(cx, cy), fontsize=10, fontweight='bold',
                    color=carrier_color,
                    bbox=dict(boxstyle='round,pad=0.2', fc='yellow', alpha=0.8),
                    ha='center', va='center', zorder=10)

# Pure silicon
draw_lattice(axes[0], 'Pure (Intrinsic) Silicon')
axes[0].text(2, -0.4, '4 bonds per atom — all electrons locked', ha='center', fontsize=9)

# N-type: Phosphorus with 5 valence electrons — one free electron
draw_lattice(axes[1], 'N-Type (Phosphorus Dopant)',
             highlight_pos=(2, 2), highlight_type='P',
             extra_carrier=(2.5, 2.5, 'Free e⁻', '#e74c3c'))
axes[1].text(2, -0.4, 'P has 5 valence e⁻ — 1 extra electron is FREE', ha='center', fontsize=9)

# P-type: Boron with 3 valence electrons — one hole
draw_lattice(axes[2], 'P-Type (Boron Dopant)',
             highlight_pos=(2, 2), highlight_type='B',
             extra_carrier=(2.5, 1.5, 'Hole (h⁺)', '#8e44ad'))
axes[2].text(2, -0.4, 'B has 3 valence e⁻ — 1 bond is EMPTY (hole)', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

## The Material Science Why — N-Type Doping

**N-type doping** adds atoms with **5 valence electrons** — typically **phosphorus (P)** or **arsenic (As)**.

What happens physically:
1. A phosphorus atom replaces one silicon atom in the lattice
2. Phosphorus forms 4 covalent bonds with its silicon neighbors (uses 4 of its 5 electrons)
3. The **5th electron** has no bond to join — it is loosely bound and easily freed
4. At room temperature, this extra electron breaks free and becomes a **free carrier**

We call phosphorus a **donor atom** because it *donates* an electron to the conduction band.

**Key insight:** The silicon is still electrically **neutral** overall. The phosphorus atom has 15 protons and 15 electrons — adding it does not add net charge. But it does add a mobile electron that can carry current.

- **Majority carriers** in N-type: electrons
- **Minority carriers** in N-type: holes (from thermal generation)

## The Material Science Why — P-Type Doping

**P-type doping** adds atoms with **3 valence electrons** — typically **boron (B)** or **gallium (Ga)**.

What happens physically:
1. A boron atom replaces one silicon atom in the lattice
2. Boron can only form **3 covalent bonds** — one bond site is **empty**
3. This empty bond is a **hole** — a place where an electron *could* be but isn't
4. A nearby electron can jump into this hole, which moves the hole to a new position

We call boron an **acceptor atom** because it *accepts* an electron from a neighboring bond.

**Key insight:** Holes behave like positive charge carriers. When an electron jumps left to fill a hole, the hole effectively moves right. We can track hole movement as if holes were real positive particles.

- **Majority carriers** in P-type: holes
- **Minority carriers** in P-type: electrons (from thermal generation)

## Concept — What Happens When P Meets N

When a piece of P-type silicon is joined to a piece of N-type silicon, something remarkable happens **at the junction**:

1. **Diffusion begins:** Electrons from the N-side (where they are abundant) diffuse across to the P-side. Holes from the P-side diffuse to the N-side.

2. **Recombination:** Electrons crossing into P-type fill holes. Holes crossing into N-type capture electrons. Near the junction, free carriers are annihilated.

3. **Exposed ions:** The donor atoms on the N-side that lost their electrons become **positive ions**. The acceptor atoms on the P-side that gained electrons become **negative ions**. These ions are fixed in the lattice — they cannot move.

4. **Electric field forms:** The positive ions on the N-side and negative ions on the P-side create an **electric field** pointing from N to P. This field opposes further diffusion.

5. **Equilibrium:** Diffusion (pushing carriers across) balances drift (the electric field pushing them back). A stable **depletion region** forms — a zone with no free carriers.

The voltage across this depletion region is the **built-in potential**:
- Silicon: **~0.6 to 0.7 V**
- Germanium: **~0.3 V**
- GaAs: **~1.2 V**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

fig, axes = plt.subplots(3, 1, figsize=(12, 12))

# --- Panel 1: PN Junction Diagram ---
ax = axes[0]
ax.set_xlim(0, 10)
ax.set_ylim(0, 4)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('PN Junction — Depletion Region Formation', fontsize=14, fontweight='bold')

# P-type region
p_region = patches.FancyBboxPatch((0.5, 0.5), 4, 3, boxstyle='round,pad=0.1',
                                   facecolor='#fadbd8', edgecolor='black', lw=2)
ax.add_patch(p_region)
ax.text(1.5, 3.0, 'P-Type', fontsize=14, fontweight='bold', color='#c0392b')

# N-type region
n_region = patches.FancyBboxPatch((5.5, 0.5), 4, 3, boxstyle='round,pad=0.1',
                                   facecolor='#d5f5e3', edgecolor='black', lw=2)
ax.add_patch(n_region)
ax.text(7.5, 3.0, 'N-Type', fontsize=14, fontweight='bold', color='#27ae60')

# Depletion region
dep_region = patches.FancyBboxPatch((3.8, 0.5), 2.4, 3, boxstyle='round,pad=0.05',
                                     facecolor='#f9e79f', edgecolor='#f39c12',
                                     lw=2, linestyle='--', alpha=0.7)
ax.add_patch(dep_region)
ax.text(5.0, 3.2, 'Depletion\nRegion', fontsize=10, fontweight='bold',
        color='#f39c12', ha='center', va='center')

# Fixed ions in depletion region
for y_pos in [1.2, 2.0, 2.6]:
    ax.text(4.2, y_pos, '\u2296', fontsize=16, color='#c0392b', ha='center')  # negative ions (P-side)
    ax.text(5.8, y_pos, '\u2295', fontsize=16, color='#27ae60', ha='center')  # positive ions (N-side)

# Holes in P-type
for pos in [(1.2, 1.5), (1.8, 2.2), (2.5, 1.0), (1.5, 2.8), (2.8, 2.5)]:
    ax.plot(*pos, 'o', color='#e74c3c', markersize=8, markeredgecolor='black', markeredgewidth=0.5)
    ax.text(pos[0], pos[1], '+', fontsize=7, ha='center', va='center', color='white', fontweight='bold')

# Electrons in N-type
for pos in [(7.0, 1.5), (7.8, 2.2), (8.5, 1.0), (7.5, 2.8), (8.2, 2.5)]:
    ax.plot(*pos, 'o', color='#2980b9', markersize=8, markeredgecolor='black', markeredgewidth=0.5)
    ax.text(pos[0], pos[1], '\u2212', fontsize=8, ha='center', va='center', color='white', fontweight='bold')

# Electric field arrow
ax.annotate('', xy=(4.1, 0.2), xytext=(5.9, 0.2),
            arrowprops=dict(arrowstyle='->', color='#8e44ad', lw=2.5))
ax.text(5.0, -0.1, 'Electric Field (E)', fontsize=10, ha='center', color='#8e44ad')

# --- Panel 2: Charge density ---
ax2 = axes[1]
x = np.linspace(-5, 5, 1000)
charge = np.piecewise(x, [
    x < -1,
    (x >= -1) & (x < 0),
    (x >= 0) & (x < 1.5),
    x >= 1.5
], [0, -1, 1, 0])

ax2.fill_between(x[x < 0], charge[x < 0], alpha=0.3, color='#e74c3c')
ax2.fill_between(x[x >= 0], charge[x >= 0], alpha=0.3, color='#2980b9')
ax2.plot(x, charge, 'k-', lw=2)
ax2.axhline(y=0, color='gray', linestyle='-', lw=0.5)
ax2.axvline(x=0, color='gray', linestyle='--', lw=0.5)
ax2.set_xlabel('Position', fontsize=12)
ax2.set_ylabel('Charge Density', fontsize=12)
ax2.set_title('Charge Density Across the Junction', fontsize=14, fontweight='bold')
ax2.text(-0.5, -0.5, 'Negative ions\n(acceptors)', fontsize=9, ha='center', color='#e74c3c')
ax2.text(0.75, 0.5, 'Positive ions\n(donors)', fontsize=9, ha='center', color='#2980b9')
ax2.set_yticks([-1, 0, 1])
ax2.set_yticklabels(['-qNₐ', '0', '+qN_d'])

# --- Panel 3: Electric potential ---
ax3 = axes[2]
x2 = np.linspace(-5, 5, 1000)
# Simple approximation of built-in potential profile
V_bi = 0.7
potential = np.piecewise(x2, [
    x2 < -1,
    (x2 >= -1) & (x2 < 0),
    (x2 >= 0) & (x2 < 1.5),
    x2 >= 1.5
], [
    0,
    lambda x: V_bi * 0.4 * (x + 1)**2,
    lambda x: V_bi * (0.4 + 0.6 * (1 - (1 - x/1.5)**2)),
    V_bi
])

ax3.plot(x2, potential, 'k-', lw=2.5)
ax3.axhline(y=0, color='gray', linestyle='-', lw=0.5)
ax3.axhline(y=V_bi, color='gray', linestyle='--', lw=0.5)
ax3.set_xlabel('Position', fontsize=12)
ax3.set_ylabel('Electric Potential (V)', fontsize=12)
ax3.set_title('Built-in Potential Across the Junction', fontsize=14, fontweight='bold')
ax3.annotate(f'V_bi = {V_bi} V', xy=(2, V_bi), xytext=(3, 0.5),
             fontsize=12, fontweight='bold',
             arrowprops=dict(arrowstyle='->', color='#e74c3c'),
             color='#e74c3c')
ax3.text(-3, 0.1, 'P-side', fontsize=12, fontweight='bold', color='#c0392b')
ax3.text(3, 0.1, 'N-side', fontsize=12, fontweight='bold', color='#27ae60')

plt.tight_layout()
plt.show()

## The Material Science Why — The Depletion Region

The depletion region is the most important concept in semiconductor physics. Here is what is physically happening:

**What it is:**
- A region at the PN junction that is **depleted of free carriers** (no mobile electrons, no mobile holes)
- Contains only **fixed ions** — positive donor ions on the N-side, negative acceptor ions on the P-side
- Behaves as an **insulator** because there are no mobile charges to carry current

**Why it matters:**
- The depletion region width determines whether current flows
- **Forward bias** (positive on P, negative on N): External voltage *opposes* the built-in potential, shrinks the depletion region, current flows
- **Reverse bias** (positive on N, negative on P): External voltage *adds to* the built-in potential, widens the depletion region, no current flows

**Width depends on doping:**
- Heavily doped side has a **narrow** depletion region (many ions packed closely)
- Lightly doped side has a **wide** depletion region (fewer ions spread farther)
- Total charge must balance: `qNₐxₚ = qN_dxₙ`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Depletion region width vs applied voltage
# W = sqrt(2 * epsilon * (V_bi - V_a) / q * (1/Na + 1/Nd))

epsilon_si = 11.7 * 8.854e-12  # permittivity of silicon (F/m)
q = 1.6e-19  # electron charge (C)
Na = 1e22  # acceptor concentration (m^-3)
Nd = 1e22  # donor concentration (m^-3)
V_bi = 0.7  # built-in potential (V)

# Applied voltage range: -10V (reverse) to +0.6V (forward, before heavy conduction)
Va = np.linspace(-10, 0.6, 500)

# Depletion width formula
W = np.sqrt(2 * epsilon_si * (V_bi - Va) / q * (1/Na + 1/Nd))
W_nm = W * 1e9  # convert to nanometers

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(Va, W_nm, 'b-', lw=2.5)
ax.axvline(x=0, color='gray', linestyle='--', lw=0.8, label='Zero bias')
ax.fill_betweenx([0, max(W_nm)*1.1], 0, 0.6, alpha=0.1, color='green', label='Forward bias')
ax.fill_betweenx([0, max(W_nm)*1.1], -10, 0, alpha=0.1, color='red', label='Reverse bias')

# Mark zero-bias width
W_0 = np.sqrt(2 * epsilon_si * V_bi / q * (1/Na + 1/Nd)) * 1e9
ax.plot(0, W_0, 'ko', markersize=10, zorder=5)
ax.annotate(f'Zero bias: W = {W_0:.0f} nm', xy=(0, W_0), xytext=(1.5, W_0 + 50),
            fontsize=11, arrowprops=dict(arrowstyle='->', color='black'))

ax.set_xlabel('Applied Voltage (V)', fontsize=13)
ax.set_ylabel('Depletion Region Width (nm)', fontsize=13)
ax.set_title('Depletion Region Width vs. Applied Voltage', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, max(W_nm) * 1.1)

plt.tight_layout()
plt.show()

print(f"At zero bias:    W = {W_0:.1f} nm")
print(f"At -5V reverse:  W = {np.sqrt(2 * epsilon_si * (V_bi + 5) / q * (1/Na + 1/Nd)) * 1e9:.1f} nm")
print(f"At -10V reverse: W = {np.sqrt(2 * epsilon_si * (V_bi + 10) / q * (1/Na + 1/Nd)) * 1e9:.1f} nm")
print(f"At +0.5V forward: W = {np.sqrt(2 * epsilon_si * (V_bi - 0.5) / q * (1/Na + 1/Nd)) * 1e9:.1f} nm")

## Concept — Forward and Reverse Bias

### Forward Bias (V_P > V_N)
- Apply **positive** to P-side, **negative** to N-side
- External voltage **opposes** the built-in potential
- Depletion region **shrinks**
- When external voltage approaches V_bi (~0.7V for Si), the barrier is low enough that carriers flood across
- Current increases **exponentially** with voltage

### Reverse Bias (V_N > V_P)
- Apply **positive** to N-side, **negative** to P-side
- External voltage **adds to** the built-in potential
- Depletion region **widens**
- Only a tiny **leakage current** flows (minority carriers swept across by the field)
- This leakage current is called the **reverse saturation current** I_s (typically nanoamps for silicon)

### Breakdown
- If reverse voltage is large enough, the junction **breaks down**
- Avalanche breakdown: carriers gain enough energy to knock more carriers free
- Zener breakdown: electric field strong enough to rip electrons from bonds
- We will study this in detail in notebook 02 (Zener diodes)

## Concept — The Shockley Diode Equation

The current through a PN junction is governed by the **Shockley diode equation**:

$$I = I_s \left( e^{\frac{V}{nV_T}} - 1 \right)$$

Where:
- **I** = diode current
- **I_s** = reverse saturation current (typically 10⁻¹² to 10⁻⁹ A for silicon)
- **V** = voltage across the junction
- **n** = ideality factor (1 for ideal, 1-2 for real diodes)
- **V_T** = thermal voltage = kT/q
  - k = Boltzmann constant = 1.381 x 10⁻²³ J/K
  - T = temperature in Kelvin
  - q = electron charge = 1.602 x 10⁻¹⁹ C
  - **At room temperature (300K): V_T ≈ 26 mV**

Key observations:
- Forward bias (V > 0): current rises **exponentially** — doubling roughly every 60mV increase
- Reverse bias (V < 0): the exponential term vanishes, I approaches **-I_s** (tiny leakage)
- Temperature increases I_s dramatically (doubles roughly every 10°C)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Shockley diode equation: I = Is * (exp(V / (n*Vt)) - 1)

k = 1.381e-23      # Boltzmann constant (J/K)
q_e = 1.602e-19    # electron charge (C)
T = 300             # temperature (K)
Vt = k * T / q_e   # thermal voltage

Is = 1e-12          # reverse saturation current (A)
n = 1.0             # ideality factor

# Voltage range
V_forward = np.linspace(0, 0.8, 1000)
V_reverse = np.linspace(-2, 0, 500)
V_full = np.concatenate([V_reverse, V_forward])

# Current calculation (clip to avoid overflow)
I_full = Is * (np.exp(np.clip(V_full / (n * Vt), -500, 500)) - 1)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# --- Linear scale ---
ax1 = axes[0]
ax1.plot(V_full * 1000, I_full * 1000, 'b-', lw=2.5)
ax1.axhline(y=0, color='gray', lw=0.5)
ax1.axvline(x=0, color='gray', lw=0.5)
ax1.set_xlabel('Voltage (mV)', fontsize=12)
ax1.set_ylabel('Current (mA)', fontsize=12)
ax1.set_title('Diode IV Curve — Linear Scale', fontsize=14, fontweight='bold')
ax1.set_xlim(-500, 800)
ax1.set_ylim(-0.5, 50)
ax1.grid(True, alpha=0.3)

# Annotate regions
ax1.annotate('Forward bias:\nExponential rise',
             xy=(700, 30), fontsize=11, fontweight='bold', color='#27ae60',
             bbox=dict(boxstyle='round', fc='#d5f5e3', alpha=0.8))
ax1.annotate('Reverse bias:\nI ≈ -Is (tiny leakage)',
             xy=(-400, 5), fontsize=11, fontweight='bold', color='#e74c3c',
             bbox=dict(boxstyle='round', fc='#fadbd8', alpha=0.8))

# Mark "knee" voltage
ax1.axvline(x=600, color='orange', linestyle='--', lw=1.5)
ax1.text(615, 40, '~0.6V "knee"', fontsize=10, color='orange')

# --- Log scale ---
ax2 = axes[1]
V_log = np.linspace(0, 0.75, 1000)
I_log = Is * (np.exp(V_log / (n * Vt)) - 1)

ax2.semilogy(V_log * 1000, I_log, 'b-', lw=2.5)
ax2.set_xlabel('Forward Voltage (mV)', fontsize=12)
ax2.set_ylabel('Forward Current (A) — Log Scale', fontsize=12)
ax2.set_title('Forward IV Curve — Log Scale', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, which='both')
ax2.set_ylim(1e-12, 1)

ax2.annotate('Straight line on log scale\n= exponential relationship',
             xy=(400, 1e-5), fontsize=11,
             bbox=dict(boxstyle='round', fc='lightyellow'))

plt.tight_layout()
plt.show()

print(f"Thermal voltage at {T}K: Vt = {Vt*1000:.2f} mV")
print(f"Current at 0.5V: {Is * (np.exp(0.5/(n*Vt)) - 1) * 1000:.4f} mA")
print(f"Current at 0.6V: {Is * (np.exp(0.6/(n*Vt)) - 1) * 1000:.2f} mA")
print(f"Current at 0.7V: {Is * (np.exp(0.7/(n*Vt)) - 1) * 1000:.1f} mA")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
try:
    import ipywidgets as widgets
    from IPython.display import display
    # Only enable widgets in interactive notebook environments
    import os
    HAS_WIDGETS = not os.environ.get('JUPYTER_EXECUTE', '') and hasattr(widgets, 'interactive_output')
    # Detect non-interactive execution (e.g., jupyter execute, nbconvert)
    try:
        ip = get_ipython()
        if hasattr(ip, 'kernel') and ip.kernel is not None:
            HAS_WIDGETS = True
        else:
            HAS_WIDGETS = False
    except NameError:
        HAS_WIDGETS = False
except ImportError:
    HAS_WIDGETS = False

def plot_iv_temperature(T=300, n=1.0, Is_exp=-12):
    """Plot diode IV curve at a given temperature."""
    k = 1.381e-23
    q_e = 1.602e-19
    Vt = k * T / q_e
    Is = 10**Is_exp
    
    V = np.linspace(0, 0.9, 1000)
    I = Is * (np.exp(np.clip(V / (n * Vt), 0, 500)) - 1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))
    
    # Linear
    ax1.plot(V, I * 1000, 'b-', lw=2.5)
    ax1.set_xlabel('Voltage (V)', fontsize=12)
    ax1.set_ylabel('Current (mA)', fontsize=12)
    ax1.set_title(f'IV Curve at T = {T}K ({T-273:.0f}°C)', fontsize=13, fontweight='bold')
    ax1.set_ylim(0, 100)
    ax1.grid(True, alpha=0.3)
    ax1.text(0.05, 80, f'Vt = {Vt*1000:.2f} mV\nIs = {Is:.1e} A\nn = {n:.1f}',
             fontsize=11, bbox=dict(boxstyle='round', fc='lightyellow'))
    
    # Log scale
    ax2.semilogy(V, I, 'b-', lw=2.5)
    ax2.set_xlabel('Voltage (V)', fontsize=12)
    ax2.set_ylabel('Current (A)', fontsize=12)
    ax2.set_title('Log Scale', fontsize=13, fontweight='bold')
    ax2.grid(True, alpha=0.3, which='both')
    ax2.set_ylim(Is * 0.1, 10)
    
    plt.tight_layout()
    plt.show()

if HAS_WIDGETS:
    temp_slider = widgets.IntSlider(value=300, min=200, max=450, step=10,
                                     description='Temp (K):', style={'description_width': 'initial'})
    n_slider = widgets.FloatSlider(value=1.0, min=1.0, max=2.0, step=0.1,
                                    description='Ideality (n):', style={'description_width': 'initial'})
    Is_slider = widgets.FloatSlider(value=-12, min=-14, max=-8, step=0.5,
                                     description='log\u2081\u2080(Is):', style={'description_width': 'initial'})
    
    out = widgets.interactive_output(plot_iv_temperature,
                                      {'T': temp_slider, 'n': n_slider, 'Is_exp': Is_slider})
    display(widgets.VBox([widgets.HBox([temp_slider, n_slider, Is_slider]), out]))
else:
    print("Running in non-interactive mode \u2014 showing static plots at 3 temperatures.")
    for T_val in [250, 300, 400]:
        plot_iv_temperature(T=T_val)

## Key Takeaways

| Concept | Key Point |
|---------|----------|
| Silicon crystal | 4 valence electrons, covalent bonds, diamond cubic |
| N-type doping | 5-electron donor (P, As) adds free electrons |
| P-type doping | 3-electron acceptor (B, Ga) creates holes |
| Depletion region | No free carriers, fixed ions, built-in electric field |
| Built-in potential | ~0.7V (Si), ~0.3V (Ge) — sets the "turn-on" voltage |
| Forward bias | Shrinks depletion region, exponential current |
| Reverse bias | Widens depletion region, tiny leakage current |
| Shockley equation | I = Is(e^(V/nVt) - 1), exponential behavior |

## Simulation

Explore the PN junction interactively in Falstad's circuit simulator:

**Diode IV Curve:**  
[https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKABcQAWEPKvPFPxYp0UJCnJQU4vmEnpkMeqmSyYEMdL4q1ITVp2yAZgEMANgGcR2-cOlHTF5gHcQ54S9EWpbjxB8-by9pNwcnPzCJARUwjxjE0NZIySlZAFk5IA](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKABcQAWEPKvPFPxYp0UJCnJQU4vmEnpkMeqmSyYEMdL4q1ITVp2yAZgEMANgGcR2-cOlHTF5gHcQ54S9EWpbjxB8-by9pNwcnPzCJARUwjxjE0NZIySlZAFk5IA)

Try:
- Sweep the voltage source from -2V to +1V and watch the current
- Notice how current stays near zero until ~0.6V, then rises steeply

## Datasheet Connection

The physics in this notebook directly maps to real datasheet parameters:

| Physics Concept | Datasheet Parameter |
|----------------|--------------------|
| Forward bias turn-on | **V_F** (Forward Voltage) — typically 0.6-0.7V for Si at rated current |
| Reverse saturation current | **I_R** (Reverse Current) — leakage in the nA to uA range |
| Reverse breakdown | **V_BR** (Breakdown Voltage) — max reverse voltage before failure |
| Temperature effects on Vt | **Thermal derating curves** — V_F decreases ~2mV/°C |
| Depletion region capacitance | **C_d** (Junction Capacitance) — changes with reverse voltage |

We will explore these in detail in notebook 03 (Reading a Diode Datasheet).

## Checkpoint Questions

Test your understanding before moving on:

1. **Why does adding phosphorus to silicon create free electrons, even though phosphorus is electrically neutral?**

2. **In N-type silicon, what are the majority carriers? What are the minority carriers?**

3. **Explain in your own words why the depletion region forms. What stops it from growing infinitely?**

4. **Why is the built-in potential ~0.7V for silicon but ~0.3V for germanium?** (Hint: think about bandgap energy)

5. **A diode has V_F = 0.65V at 1mA. Using the Shockley equation, what is the approximate current at 0.71V?** (Assume n=1, T=300K. Hint: each 60mV roughly doubles the current)

6. **If you increase the temperature of a diode, does the forward voltage at a fixed current increase or decrease? Why?**

7. **What physical mechanism causes the reverse saturation current I_s?**

---

*Next notebook: [01-diodes-in-practice.ipynb](01-diodes-in-practice.ipynb) — Real diode experiments with 1N4148, 1N5817, and LEDs*